In [1]:
def mask_generate(result,level,img_name,generate_path):
    level_list=[]
    if str(1) in level:
        level_list= [11,12]
    if str(2) in level:
        level_list = level_list + [13,14,15,16,17,18]
    if str(3) in level:
        level_list = level_list + [4,5,6,7]
        
    print(level_list)
    [rows,cols] = result[0].shape
    thre=5
    
    for i in range(rows):
        for j in range(cols):
            if result[0][i,j] in level_list:
                result[0][i,j]=255
                for m in range(thre*2+1):
                    for n in range(thre*2+1):
                        num_x = i-m
                        num_y = j-n
                        if (num_x >= 0) & (num_y >= 0) & (num_x<rows) & (num_y<cols):
                             result[0][num_x,num_y] = 255
                            
            else:
                result[0][i,j]=0
                
    output_path = os.path.join(generate_path,img_path.split('/')[-1])
    image = Image.fromarray(result[0].astype('uint8'))
    image.save(output_path)

In [3]:
def mask_select(result,level_list,img_path,sod_path):

    level_dict = {}

    saliency = os.path.splitext(img_path)[0]+'.jpg'
    saliecny = os.path.join(sod_path,saliency)
    img = Image.open(saliency).convert('L')
    img_np = np.array(img)
    
    [rows,cols] = result[0].shape
    num_pix = rows * cols
    max_value = img_np.max()
    
    num = 0
    for i in range(rows):
        for j in range(cols):       
            if result[0][i,j] in level_list:
                num += 1
                if img_np[i,j]>=0.8*max_value:
                    if result[0][i,j] not in level_dict.keys():
                        
                        level_dict[str(result[0][i,j])] = 1
                    else:
                        level_dict[str(result[0][i,j])] += 1
    
    output_list = []
    for key,value in level_dict.items():
        if int(key) in level_1:
            if str(1) not in output_list: 
                output_list.append("1")
        elif int(key) in level_2:
            if str(2) not in output_list: 
                output_list.append("2")
        elif int(key) in level_3:
            if str(3) not in output_list: 
                output_list.append("3")
                
    return output_list

In [ ]:
from mmseg.apis import init_segmentor, inference_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette
import os
from PIL import Image
import numpy as np

level_1 = [11,12]                 #cityscape dataset label person
level_2 = [13,14,15,16,17,18]     #cityscape dataset label veichles
level_3 = [4,5,6,7]               #cityscape dataset label fence,pole,traffic light, and traffic sign
level_all = level_1 + level_2 + level_3

data_path = '../../dataset/raw_imgs'
mask_path = '../../dataset/masks'
sod_path = '../../dataset/sod_u2net'

config_file = '../../configs/deeplabv3plus/deeplabv3plus_r101-d8_769x769_80k_cityscapes.py'
checkpoint_file = '../../checkpoints/deeplabv3plus_r101-d8_769x769_80k_cityscapes_20200607_000405-a7573d20.pth'

model = init_segmentor(config_file, checkpoint_file, device='cuda:0')

images = os.listdir(data_path)
for i in images:
    
    img_path = os.path.join(data_path, i)
    
    # image segmentation
    result = inference_segmentor(model, img_path)
    
    # saliency-based mask selection considering three levels segmentation
    mask_list = mask_select(result, level_all, img_path, sod_path)

    # mask generator
    mask = mask_generate(result,mask_list,img_path,mask_path)

load checkpoint from local path: ../../checkpoints/deeplabv3plus_r101-d8_769x769_80k_cityscapes_20200607_000405-a7573d20.pth
